In [1]:
#from sklearn.datasets import fetch_20newsgroups
#remove=("headers","footers","quotes")
#texts=fetch_20newsgroups(subset="train",remove=remove).data

In [2]:
from __future__ import print_function
import numpy as np

In [3]:
docs=['turkey day holiday',
          'cake holiday',
          'turkey race thanksgiving holiday',
          'snail race turtle',
          'time travel space race',
          'movie thanksgiving',
          'movie air space museum cool movie',
          'aspiring movie star']
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father"
doc2 = "My father spends a lot of time driving my sister around to dance practice"
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure"
#doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better"
doc5 = "Health experts say that Sugar is not good for your lifestyle"

# compile documents
doc_complete = [doc1, doc2, doc3,  doc5]

#docs=docs+doc_complete

In [4]:
docs_struct=[]
for i in range(len(docs)):
    docs_struct.append(docs[i].split(" "))

In [5]:
docs_struct

[['turkey', 'day', 'holiday'],
 ['cake', 'holiday'],
 ['turkey', 'race', 'thanksgiving', 'holiday'],
 ['snail', 'race', 'turtle'],
 ['time', 'travel', 'space', 'race'],
 ['movie', 'thanksgiving'],
 ['movie', 'air', 'space', 'museum', 'cool', 'movie'],
 ['aspiring', 'movie', 'star']]

In [6]:
n_topics=3

In [7]:
g=np.array([])
for i in range(len(docs_struct)):
    g=np.append(g,docs_struct[i])

In [8]:
vocab=np.unique(g)
print(vocab.shape)
vocab

(17,)


array(['air', 'aspiring', 'cake', 'cool', 'day', 'holiday', 'movie',
       'museum', 'race', 'snail', 'space', 'star', 'thanksgiving', 'time',
       'travel', 'turkey', 'turtle'], dtype='<U32')

In [9]:
word_topic_count=np.zeros([n_topics,len(vocab)])
topic_alloc=[]
docs_pertopic_count=[]
for i in range(len(docs_struct)):
    topic_alloc.append([])
    docs_pertopic_count.append(np.zeros(n_topics))
    for j in range(len(docs_struct[i])):
        for k in range(len(vocab)):
            if(vocab[k]==docs_struct[i][j]):
                docs_struct[i][j]=(k+1)
                
                topic_alloc[i].append(np.random.randint(n_topics))
                
                topic_index=topic_alloc[i][j]
                word_index=docs_struct[i][j]
                word_topic_count[topic_index][word_index-1]+=1
                
                docs_pertopic_count[i][topic_index]+=1

In [10]:
docs_struct

[[16, 5, 6],
 [3, 6],
 [16, 9, 13, 6],
 [10, 9, 17],
 [14, 15, 11, 9],
 [7, 13],
 [7, 1, 11, 8, 4, 7],
 [2, 7, 12]]

In [11]:
word_topic_count=np.array(word_topic_count)
word_topic_count
print(word_topic_count)

[[0. 1. 1. 0. 1. 0. 4. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 2. 0. 1. 1. 1. 2. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 2. 0.]]


In [12]:
topic_alloc

[[2, 0, 1],
 [0, 2],
 [2, 0, 0, 1],
 [1, 1, 0],
 [2, 2, 1, 2],
 [0, 2],
 [0, 2, 1, 1, 2, 0],
 [0, 0, 2]]

In [13]:
docs_pertopic_count=np.array(docs_pertopic_count)
docs_pertopic_count

array([[1., 1., 1.],
       [1., 0., 1.],
       [2., 1., 1.],
       [1., 2., 0.],
       [0., 1., 3.],
       [1., 0., 1.],
       [2., 2., 2.],
       [2., 0., 1.]])

In [14]:
alpha=1
beta=0.1
iterations=30
for i in range(iterations):
    for j in range(len(docs_struct)):
        for k in range(len(docs_struct[j])):
            topic=topic_alloc[j][k]
            word=docs_struct[j][k]
            
            docs_pertopic_count[j][topic]-=1
            word_topic_count[topic][word-1]-=1
            p=[]
            for t in range(n_topics):
                indoc_topic_count=docs_pertopic_count[j][t]+alpha
                indoc_alltopics_count=sum(docs_pertopic_count[j])+n_topics*alpha
            
                alldoc_word_topic_count=word_topic_count[t][word-1]+beta
                alldoc_word_alltopic_count=np.sum(word_topic_count,axis=0)[word-1]+len(vocab)*beta
                p.append((indoc_topic_count/indoc_alltopics_count)*(alldoc_word_topic_count/alldoc_word_alltopic_count))
            p=p/sum(p)
            new_topic=np.argmax(p)
            if topic!=new_topic:
                print(topic,p,new_topic)
            docs_pertopic_count[j][new_topic]+=1
            word_topic_count[new_topic][word-1]+=1
            topic_alloc[j][k]=new_topic

0 [0.2 0.4 0.4] 1
0 [0.25 0.25 0.5 ] 2
2 [0.04166667 0.875      0.08333333] 1
0 [0.04347826 0.47826087 0.47826087] 1
0 [0.03846154 0.11538462 0.84615385] 2
1 [0.4 0.4 0.2] 0
2 [0.02173913 0.91304348 0.06521739] 1
2 [0.375 0.375 0.25 ] 0
1 [0.5  0.25 0.25] 0
2 [0.625 0.25  0.125] 0
2 [0.6 0.2 0.2] 0
2 [0.25 0.5  0.25] 1
2 [0.16666667 0.5        0.33333333] 1
2 [0.16666667 0.66666667 0.16666667] 1


In [15]:
docs_pertopic_count

array([[0., 2., 1.],
       [0., 2., 0.],
       [0., 2., 2.],
       [2., 1., 0.],
       [0., 4., 0.],
       [1., 0., 1.],
       [5., 1., 0.],
       [3., 0., 0.]])

In [17]:
topic1=[]
topic2=[]
topic3=[]
for j in range(len(docs_struct)):
        for k in range(len(docs_struct[j])):
            topic=topic_alloc[j][k]
            word=vocab[docs_struct[j][k]-1]
            if topic==0:
                topic1.append(word)
            elif topic==1:
                topic2.append(word)
            elif topic==2:
                topic3.append(word)
                
print(np.unique(np.array(topic1)))
print(np.unique(np.array(topic2)))
print(np.unique(np.array(topic3)))

['air' 'aspiring' 'cool' 'movie' 'museum' 'snail' 'star' 'turtle']
['cake' 'day' 'holiday' 'race' 'space' 'time' 'travel']
['thanksgiving' 'turkey']
